In [30]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [31]:
import sklearn.datasets as datasets
from sklearn.model_selection import train_test_split

In [32]:
import pandas as pd

In [33]:
wine = datasets.load_wine()

In [34]:
print(wine.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [35]:
wine_data = wine.data[0:130]
wine_target = wine.target[0:130]

train_X, test_X, train_Y, test_Y = train_test_split(wine_data, wine_target, test_size=0.25)

In [36]:
len(train_X)

97

In [37]:
len(test_X)

33

In [38]:
train_Xt = torch.from_numpy(train_X).float()
train_Yt = torch.from_numpy(train_Y).long()

test_Xt = torch.from_numpy(test_X).float()
test_Yt = torch.from_numpy(test_Y).long()

In [39]:
train = TensorDataset(train_Xt, train_Yt)

In [40]:
train[0]

(tensor([1.2000e+01, 3.4300e+00, 2.0000e+00, 1.9000e+01, 8.7000e+01, 2.0000e+00,
         1.6400e+00, 3.7000e-01, 1.8700e+00, 1.2800e+00, 9.3000e-01, 3.0500e+00,
         5.6400e+02]), tensor(1))

In [41]:
train_loader = DataLoader(train, batch_size=15, shuffle=True)

In [54]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=0)

In [55]:
model = Net()

In [56]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [57]:
for epoch in range(500):
    total_loss = 0
    
    for train_x, train_y in train_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        optimizer.zero_grad()
        output = model(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if (epoch+1)%60 == 0:
        print(epoch+1, total_loss)

60 1.749649291858077
120 1.2785241156816483
180 1.3817394748330116
240 1.173574410378933
300 0.974313709884882
360 1.458410881459713
420 0.8840397447347641
480 1.1213245689868927


In [58]:
test_x, test_y = Variable(test_Xt), Variable(test_Yt)
result = torch.max(model(test_x).data, 1)[1]
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

In [59]:
accuracy

0.8787878787878788

In [28]:
sum(test_y.data.numpy() == result.numpy())

36

In [29]:
len(test_y)

39